<a href="https://colab.research.google.com/github/wiesehahn/waldmaske/blob/master/notebooks/ee_tensorflow_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Copyright 2020 Wiesehahn { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/wiesehahn/waldmaske/blob/master/notebooks/ee_tensorflow_mask.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/wiesehahn/waldmaske/blob/master/notebooks/ee_tensorflow_mask.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow notebook. 

<img src=https://findicons.com/files/icons/2781/google_jfk_icons/128/earth_engine.png width=72>
<img src=https://docs.ray.io/en/master/_images/tensorflow.png width=72>

It is heavily inspired and adapted from [this](https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb) and [this](http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb) demonstration notebooks. It demonstrates a per-pixel neural network implemented in a way that allows the trained model to be hosted on [Google AI Platform](https://cloud.google.com/ai-platform) and used in Earth Engine for interactive prediction from an `ee.Model.fromAIPlatformPredictor`.

 

**Running this demo may incur charges to your Google Cloud Account!**

# Setup software libraries

Import software libraries and/or authenticate as necessary.

## Authenticate to Colab and Cloud

Google Cloud Storage bucket will serve as a bridge between GEE and Colab. To read/write from a Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself).  *This should be the same account you use to login to Earth Engine*.  When you run the code below, it will display a link in the output to an authentication page in your browser.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.  Copy the code from the permissions page back into this notebook and press return to complete the process.

(You may need to run this again if you get a credentials error later.)

In [0]:
from google.colab import auth
auth.authenticate_user()

## Authenticate to Earth Engine

Authenticate to Earth Engine the same way you did to the Colab notebook.  Specifically, run the code to display a link to a permissions page.  This gives you access to your Earth Engine account.  *This should be the same account you used to login to Cloud previously*.  Copy the code from the Earth Engine permissions page back into the notebook and press return to complete the process.

In [0]:
import ee
ee.Authenticate()
ee.Initialize()

## Initialize Geemap 

Import [Geemap package](https://github.com/giswqs/geemap)

(A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets.)

Note that Google Colab currently does not support ipyleaflet. Therefore, you should use import geemap.eefolium instead of import geemap.

In [0]:
!pip install geemap
import geemap.eefolium as emap

## Test the TensorFlow installation

Import TensorFlow and check the version.

In [0]:
import tensorflow as tf
print(tf.__version__)

## Test the Folium installation

We might use the Folium library for visualization where Geemap does not work.  Import the library and check the version.

In [0]:
import folium
print(folium.__version__)

# Define variables


In [0]:
# Your Earth Engine username. This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'wiesehahn'

# Replace with your Google Cloud Project
PROJECT = 'forest-201911'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written. You must be able to write into this bucket.
OUTPUT_BUCKET = 'gee_forest-mask'

# File names for the training and testing datasets. These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_lucas'
TEST_FILE_PREFIX = 'Testing_lucas'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# The name and path of the Earth Engine asset to be created 
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/waldmaske/classification/map/tf_prob_lucas_v0'


# Use these bands for prediction. B* represent S2 reflectance values for corresponding 
# bands in spring, and B*_1 represent reflectance values in summer.
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 
         'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B8_1', 'B8A_1', 'B9_1', 'B11_1', 'B12_1']

# This is a training/testing dataset of points with known land cover labels.
LABEL_DATA = 'users/wiesehahn/waldmaske/classification/reference/lucas_filtered'
# The labels, consecutive integer indices starting from zero, are stored in this property, set on each point.
LABEL = 'class'
# Number of label values, i.e. number of classes in the classification (here 'forest'/'no-forest')
N_CLASSES = 2

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# List of fixed-length features, all of which are float32.
columns = [tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES]

# Dictionary with feature names as keys, fixed-length features as values.
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))


# define region of interest
# General ROI is Germany while test area for prediction is area around Göttingen
GAUL = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")  
DE = GAUL.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
ROI = DE

TESTAREA = ee.Geometry.Polygon(
        [[[9.940686323657957, 51.5633863622234],
          [9.940686323657957, 51.504442609479455],
          [10.057416059986082, 51.504442609479455],
          [10.057416059986082, 51.5633863622234]]])


# Create Training and Testing data

Public available data from the 'Land Use and Coverage Area frame Survey' ([LUCAS](https://ec.europa.eu/eurostat/web/lucas/overview)) for 2018 was used as reference data. In order to use it for this project within Earth Engine the data had to be filtered and uploaded to EE as an asset. To download and filter the data similar to [Weigand et. al, 2020](https://www.sciencedirect.com/science/article/pii/S0303243419307317) a Script was applied in R ([script](https://github.com/wiesehahn/waldmaske/blob/master/scripts/lucas_filtered.R)).

With this Reference data a model is built based on Sentinel-2 predictor variables (`BANDS`) from two time periods.


## Load reference data

Load the labeled points from existing Earth Engine asset.  Each point in this table has a property called `class` that stores the label, encoded as an integer.

In [0]:
ref_raw = ee.Collection.loadTable(LABEL_DATA)
ref_raw = ref_raw.filterBounds(ROI)
label = LABEL

print('number of reference points:', ref_raw.size().getInfo());

In [0]:
Map = emap.Map()
Map.centerObject(ROI, 8)


# Adds Earth Engine layers to Map
Map.addLayer(TESTAREA,{'color': 'FF0000'},'test area', True, 0.3)
Map.addLayer(ref_raw, {'color': 'FFFFFF'}, 'reference', True, 0.5)

# Display the Map
Map.addLayerControl()
Map

## Prepare Sentinel-2 imagery

Create a cloud-masked composite of Sentinel-2 surface reflectance imagery from 2019 and 2020. Actually, two composites are generated for spring (doy 75-150) and summer (doy 150-270) which are then combined. In a pre-study the predictive power of different seasons was examined for a random forest classifier [see here](https://code.earthengine.google.com/569e88cfdd90edb1b66930f61b8867c2?noload=true). The results suggested that spring and summer are more suitable than winter and autumn. Although this might be different in another model we kept these variables as it is hard to get entirely cloud-free composites for autumn and winter seasons.


In [0]:
# Function to mask clouds using SCL and MSK_CLDPRB bands
def maskClouds_opt2(image):
  # ESA Cloud Probability Map
  cloudProb = image.select('MSK_CLDPRB')
  # ESA Scene Classification
  scl = image.select('SCL')
  shadow = scl.eq(3)
  cirrus = scl.eq(10)
  mask = cloudProb.lt(5).And((cirrus).neq(1)).And((shadow).neq(1))
  return image.updateMask(mask)

# Load Sentinel-2 surface reflectance data 
s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
      .filter(ee.Filter.calendarRange(2019,2020, 'year'))
      .filterBounds(ROI)
      .map(maskClouds_opt2)
      .select('B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'))

# reduce images by season  
spring = (s2
          .filter(ee.Filter.calendarRange(75, 150, 'day_of_year'))
          .reduce(ee.Reducer.percentile([33]))
          .rename('B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'))

summer = (s2
          .filter(ee.Filter.calendarRange(150, 270, 'day_of_year'))
          .reduce(ee.Reducer.percentile([33]))
          .rename('B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'))

# merge in one image
composite = spring.addBands(summer)


Display predictor images as RGB (this might take a long tim to compute!)

In [0]:
# Map = emap.Map()
# Map.centerObject(TESTAREA, 13)

# visParams ={'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1500}

# # Adds Earth Engine layers to Map
# Map.addLayer(spring, visParams,'spring', True, 1)
# Map.addLayer(summer, visParams, 'summer', True, 1)

# # Display the Map
# Map.addLayerControl()
# Map

## Add pixel values of the composite to labeled points

Here we overlay the points on imagery to get predictor variables along with labels.

In [0]:
sample = composite.sampleRegions( 
   collection= ref_raw,
   properties= [LABEL],
   scale= 10,
   tileScale= 16,
   geometries= True
).randomColumn('random')

# remap values in two classes (forest/ no-forest)
reference = sample.remap(
  # 0:T3 (High, seasonal and perennial veg.), 1:A (Artificial land), 2:S (Open soil), 3:T1 (High, seasonal veg.), 
  # 4:T2 (	High perennial veg.), 5:V1 (Low, seasonal veg.), 6:V2 (Low, perennial veg.), 7:W (Water areas)
  lookupIn=  [0,1,2,3,4,5,6,7], 
  # [0:Forest, 1:No-Forest]
  lookupOut= [0,1,1,0,0,1,1,1],
  columnName= 'class')

split = 0.7 # 70% training, 30% validation.
training = reference.filter(ee.Filter.lt('random', split))
testing = reference.filter(ee.Filter.gte('random', split))


## Export the training and testing data

Now that there's training and testing data , it's time to materialize the datasets in a place where the TensorFlow model has access to them (Unfortunately, you cannot use Tensorflow directly in Earth Engine). You can do that by exporting the training and testing datasets to tables in TFRecord format ([learn more about TFRecord format](https://www.tensorflow.org/tutorials/load_data/tf-records)) in your Cloud Storage bucket, since both GEE and GCS and Tensorflow can access to them.

In [0]:
# Make sure you can see the output bucket. You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Can not find output Cloud Storage bucket.')

Once you've verified the existence of the intended output bucket, run the exports.

In [0]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [0]:
# Start the tasks.
if tf.io.gfile.exists(TRAIN_FILE_PATH):
  print('Train file already exists')
else:
  training_task.start()

if tf.io.gfile.exists(TEST_FILE_PATH):
  print('Test file already exists')
else:
  testing_task.start()

**Monitor task progress** 

(Make sure the training and testing tasks are completed before continuing)

In [0]:
# Print all tasks.
pprint(ee.batch.Task.list())

Check existence of the exported files 

(If you've seen the status of the export tasks change to `COMPLETED`, then check for the existince of the files in the output Cloud Storage bucket)

# Read data

### Check existence of the data files

Check that you have permission to read the files in the output Cloud Storage bucket.

In [0]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No testing file found.')

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/data) explain more about reading data into a `tf.data.Dataset`).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.  Note that we will use all data for training.


In [0]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset([TRAIN_FILE_PATH],
                                        compression_type='GZIP')

# Print the first record to check.
print(iter(train_dataset).next())

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized `Example` proto (i.e. [`example.proto`](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/example/example.proto)) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.  ([Learn more about parsing `Example` protocol buffer messages](https://www.tensorflow.org/programmers_guide/datasets#parsing_tfexample_protocol_buffer_messages)).

In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by FEATURES_DICT.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the LABEL, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=4)

from pprint import pprint

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands names for keys and tensors storing the pixel data for values.  The second element of the tuple is tensor storing the class label.

## Create additional features

Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands.  Here are some helper functions for that.

In [0]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalized_difference(features['B5'], features['B4'])
  features['NDVI_1'] = normalized_difference(features['B5_1'], features['B4_1'])
  return features, label

In [0]:
# Add NDVI.
# parsed_dataset = parsed_dataset.map(add_NDVI)

## Adjust dimension and shape

Turn the dictionary of *{name: tensor,...}* into a 1x1xP array of values, where P is the number of predictors.  Turn the label into a 1x1x`N_CLASSES` array of indicators (i.e. one-hot vector), in order to use a categorical crossentropy-loss function.  Return a tuple of (predictors, indicators where each is a three dimensional array; the first two dimensions are spatial x, y (i.e. 1x1 kernel).

In [0]:
# Inputs as a tuple.  Make predictors 1x1xP and labels 1x1xN_CLASSES.
def to_tuple(inputs, label):
  return (tf.expand_dims(tf.transpose(list(inputs.values())), 1),
          tf.expand_dims(tf.one_hot(indices=label, depth=N_CLASSES), 1))

input_dataset = parsed_dataset.map(to_tuple)
# Check the first one.
pprint(iter(input_dataset).next())

input_dataset = input_dataset.shuffle(128).batch(8)

# Model setup

Make a densely-connected convolutional model, where the convolution occurs in a 1x1 kernel.  This is exactly analagous to the model generated in [this example notebook](http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb), but operates in a convolutional manner in a 1x1 kernel.  This allows Earth Engine to apply the model spatially, as demonstrated below.

Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

## Create the Keras model

Before we create the model, there's still a wee bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss.  Specifically, Keras expects a list of inputs and a one-hot vector for the class. (See [the Keras loss function docs](https://keras.io/losses/), [the TensorFlow categorical identity docs](https://www.tensorflow.org/guide/feature_columns#categorical_identity_column) and [the `tf.one_hot` docs](https://www.tensorflow.org/api_docs/python/tf/one_hot) for details).

Here we will use a simple neural network model with a 64 node hidden layer.  Once the dataset has been prepared, define the model, compile it, fit it to the training data.  See [the Keras `Sequential` model guide](https://keras.io/getting-started/sequential-model-guide/) for more details.

In [0]:
from tensorflow import keras

# Define the layers in the model.  Note the 1x1 kernels.
model = tf.keras.models.Sequential([
  #change to number of input bands
  tf.keras.layers.Input((None, None, 22,)), 
  tf.keras.layers.Conv2D(64, (1,1), activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(N_CLASSES, (1,1), activation=tf.nn.softmax)
])

# Compile the model with the specified loss and optimizer functions.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data. 
model.fit(x=input_dataset, epochs=10)


## Save the trained model

Export the trained model to TensorFlow `SavedModel` format in your cloud storage bucket.  The [Cloud Platform storage browser](https://console.cloud.google.com/storage/browser) is useful for checking on these saved models.

In [0]:
MODEL_DIR = 'gs://' + OUTPUT_BUCKET + '/lucas_pixel_model'
model.save(MODEL_DIR, save_format='tf')

# EEification

EEIfication prepares the model for hosting on [Google AI Platform](https://cloud.google.com/ai-platform).  Learn more about EEification from [this doc](https://developers.google.com/earth-engine/tensorflow#interacting-with-models-hosted-on-ai-platform).  First, get (and SET) input and output names of the nodes.  **CHANGE THE OUTPUT NAME TO SOMETHING THAT MAKES SENSE FOR YOUR MODEL!**  Keep the input name of 'array', which is how you'll pass data into the model (as an array image).

In [0]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "lucas"}) + "'"
print(input_dict)
print(output_dict)

## Run the EEifier

The actual EEification is handled by the `earthengine model prepare` command.  Note that you will need to set your Cloud Project prior to running the command.

In [0]:
# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_pixel_model'

if tf.io.gfile.exists(EEIFIED_DIR):
  print('EEified model exists already')
else:
  # You need to set the project before using the model prepare command.
  !earthengine set_project {PROJECT}
  !earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

# Deploy and host the EEified model on AI Platform

Now there is another TensorFlow `SavedModel` stored in `EEIFIED_DIR` ready for hosting by AI Platform.  Do that from the `gcloud` command line tool, installed in the Colab runtime by default.  Note that the `MODEL_NAME` must be unique.  If you already have a model by that name, either name a new model or a new version of the old model.  The [Cloud Console AI Platform models page](https://console.cloud.google.com/ai-platform/models) is useful for monitoring your models.

**If you change anything about the trained model, you'll need to re-EEify it and create a new version!**

In [0]:
MODEL_NAME = 'pixel_lucas_model'
VERSION_NAME = 'v02'

In [0]:
!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.1 \
  --python-version=3.7

# Make predictions
1. load input image
2. Connect to the hosted model.
3. Use the model to make predictions.


Note that it takes the model a couple minutes to spin up and make predictions.

In [0]:
# Turn into an array image for input to the model.
array_image = composite.float().toArray()

# Point to the model hosted on AI Platform.
model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=[8, 8],
    # # Keep this the same as your training data.
    # proj=ee.Projection('EPSG:4326').atScale(10),
    # fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'lucas': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

# model.predictImage outputs a one dimensional array image that
# packs the output nodes of your model into an array.  These
# are class probabilities that you need to unpack into a 
# multiband image with arrayFlatten().  If you want class
# labels, use arrayArgmax() as follows.
predictions = model.predictImage(array_image)
probabilities = predictions.arrayFlatten([['forest', 'no-forest']])
label = predictions.arrayArgmax().arrayGet([0]).rename('label')

#create an export task
export_task = ee.batch.Export.image.toAsset(
    image= probabilities,
    description= 'Probability Export', 
    assetId= OUTPUT_ASSET_ID,
    region = TESTAREA, 
    scale = 10)



In [0]:
# Start the export task
export_task.start()

**Monitor task progress**


In [0]:
# Print all tasks.
pprint(ee.batch.Task.list())

import time

while export_task.active():
  print('Polling for task (id: {}).'.format(export_task.id))
  time.sleep(60)
print('Done with image export.')

# Display Results

In [0]:
# load classified image and get info
predictions_image = ee.Image(OUTPUT_ASSET_ID)
print(predictions_image.bandNames().getInfo())

In [0]:
Map = emap.Map()
Map.centerObject(TESTAREA, 13)

# Sets visualization parameters
probability_vis = {'bands': ['no-forest','forest'], 'max': 0.5}
visParams ={'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1500}

# Adds Earth Engine layers to Map
Map.addLayer(TESTAREA,{},'test area', True, 0.3)
Map.addLayer(predictions_image, probability_vis, 'Forest probability', True, 0.7)
Map.addLayer(composite, visParams, 'Spring RGB (input image)', True, 0.7)


# Display the Map
Map.addLayerControl()
Map